# Publish Healthcare Fraud Dataset to Databricks Marketplace
Uses environment variables for workspace URL and PAT. Creates a Delta table and prepares listing payload.


In [ ]:
import os, json
import pandas as pd

# df should be available (generated or loaded). Example:
# from healthcare_fraud_generate import gen_claims
# df = gen_claims(200_000, fraud_rate=0.03)

workspace_api = os.getenv('VITE_DATABRICKS_API_URL')
pat = os.getenv('VITE_DATABRICKS_API_KEY')
print('Workspace API:', workspace_api)
print('PAT configured:', bool(pat))

listing = {
    'dataset_name': 'AethergenAI – Healthcare Claims Fraud v1',
    'version': '1.0.0',
    'provider': 'AUSPEXI',
    'description': 'Synthetic healthcare claims with injected fraud patterns (3% prevalence).',
    'format': 'delta_table',
    'partitions': ['year','month'],
    'target_table': 'healthcare_synth_v1',
}
print(json.dumps(listing, indent=2))


In [ ]:
# Prepare SQL to create share and add table (run on Databricks SQL)
sql_commands = f'''
CREATE SHARE IF NOT EXISTS aethergen_healthcare_v1;
ALTER SHARE aethergen_healthcare_v1 ADD TABLE aethergen.healthcare_synth_v1;
-- GRANT SELECT ON SHARE aethergen_healthcare_v1 TO RECIPIENT <your_recipient_name>;
'''
print(sql_commands)


In [ ]:
# Automate share + table add via Databricks REST (SQL Statement Execution API)
import os, time, requests, json

api_base = os.getenv('VITE_DATABRICKS_API_URL')  # e.g., https://.../api/2.0
pat = os.getenv('VITE_DATABRICKS_API_KEY')
assert api_base and pat, 'Set VITE_DATABRICKS_API_URL and VITE_DATABRICKS_API_KEY'

headers = { 'Authorization': f'Bearer {pat}', 'Content-Type': 'application/json' }

# 1) Pick a running SQL warehouse
wh_list = requests.get(f"{api_base}/sql/warehouses", headers=headers).json()
warehouses = wh_list.get('warehouses', [])
if not warehouses:
    raise RuntimeError('No SQL Warehouses found. Create/Start a Serverless Starter Warehouse and re-run.')

# Prefer RUNNING, else first
warehouse = next((w for w in warehouses if w.get('state')=='RUNNING'), warehouses[0])
warehouse_id = warehouse['id']
print('Using warehouse:', warehouse.get('name'), warehouse_id, warehouse.get('state'))

# 2) Helper to run SQL

def run_sql(sql):
    payload = { 'statement': sql, 'warehouse_id': warehouse_id }
    resp = requests.post(f"{api_base}/sql/statements", headers=headers, data=json.dumps(payload))
    resp.raise_for_status()
    sid = resp.json()['statement_id']
    # poll
    while True:
        q = requests.get(f"{api_base}/sql/statements/{sid}", headers=headers)
        q.raise_for_status()
        s = q.json()
        if s['status']['state'] in ('SUCCEEDED','FAILED','CANCELED'):
            return s
        time.sleep(1.0)

# 3) SQL for catalog/schema and share
fully_qualified_table = 'hive_metastore.aethergen.healthcare_synth_v1'
sqls = [
    "CREATE SCHEMA IF NOT EXISTS hive_metastore.aethergen",
    f"ALTER TABLE {fully_qualified_table} SET TBLPROPERTIES(delta.autoOptimize.optimizeWrite = true)",
    "CREATE SHARE IF NOT EXISTS aethergen_healthcare_v1",
    f"ALTER SHARE aethergen_healthcare_v1 ADD TABLE {fully_qualified_table}"
]

results = []
for s in sqls:
    print('Executing:', s)
    r = run_sql(s)
    print('  ->', r['status']['state'])
    results.append((s, r['status']['state']))

print('Done. Review share aethergen_healthcare_v1 and create a Recipient to grant access.')
